In [178]:
import math
from collections import defaultdict
counts = defaultdict(int)
context_counts = defaultdict(int)

In [179]:
# テスト
# training_file = open("../test/02-train-input.txt","r")

# 本番
training_file = open("../data/wiki-en-train.word","r")
test_file = open("../data/wiki-en-test.word","r")
training = training_file.readlines()
test = test_file.readlines()

In [183]:
probabilities = defaultdict(float)
for line in training:
    words = line.split()
    words.append("</s>")
    words.insert(0,"<s>")
    for i in xrange(1,len(words)):
        counts[words[i-1]+" "+words[i]] += 1
        context_counts[words[i-1]] += 1
        counts[words[i]] += 1
        context_counts[""] += 1
for ngram,count in counts.items():
    words = ngram.split()[:-1]
    context = "".join(words)
    probability = float(counts[ngram])/context_counts[context]
    probabilities[ngram] = probability

In [181]:
# Grid Search
entropies = defaultdict(float)
for lambda_1 in [0.05*i for i in xrange(1,20)]:
    for lambda_2 in [0.05*i for i in xrange(1,20)]:
        V = 1000000; W = 0; H = 0
        for line in test:
            words = line.split()
            words.append("</s>")
            words.insert(0,"<s>")
            for i in xrange(1,len(words)):
                P_1 = lambda_1*probabilities[words[i]]+(1-lambda_1)/V
                P_2 = lambda_2*probabilities[words[i-1]+" "+words[i]]+(1-lambda_2)*P_1
                H += -math.log(P_2,2)
                W += 1
        entropies[str(lambda_1)+","+str(lambda_2)] = H/W

In [182]:
for parameter,entropy in sorted(entropies.items(), key=lambda x: x[1]):
    print entropy,parameter
    break

9.66386972605 0.85,0.35


In [184]:
# Witten-Bell平滑化
unique_words = defaultdict(set)
for line in training:
    words = line.split()
    words.append("</s>")
    words.insert(0,"<s>")
    for i in xrange(1,len(words)):
        unique_words[words[i-1]].add(words[i])

entropies = defaultdict(float)
for lambda_1 in [0.05*i for i in xrange(1,20)]:
    for line in test:
        words = line.split()
        words.append("</s>")
        words.insert(0,"<s>")
        for word in words:
            for i in xrange(1,len(words)):
                u = len(unique_words[words[i-1]])
                c = counts[words[i-1]]
                lambda_2 = 1-float(u)/(u+c) if u+c!=0 else 0
                P_1 = lambda_1*probabilities[words[i]]+(1-lambda_1)/V
                P_2 = lambda_2*probabilities[words[i-1]+" "+words[i]]+(1-lambda_2)*P_1
                H += -math.log(P_2,2)
                W += 1
    entropies[str(lambda_1)] = H/W

In [185]:
for parameter,entropy in sorted(entropies.items(), key=lambda x: x[1]):
    print entropy,parameter
    break

10.3142359625 0.95
